Para hacer clasificación lineal podemos usar algoritmos diferentes a los vistos anteriormente. Entre estos algoritmos encontramos la regresión logística. A pesar del nombre, NO sirve para hacer regresión y solo se usa para clasificación.

Normalmente se usa para hacer clasificación binaria aunque se puede extender (la regresión logística y otros algoritmos de clasificación binaria) a clasificación multiclase como luego veremos.

Este caso es similar a lo que vimos anteriormente, tenemos lo siguiente:

$$ \hat{y}(x) = w_0 · x_0 + w_1 · x_1 + ... + w_n · x_n $$

Aquí $ w_0 = b $ y $ x_0 = 1 $.

Y nuestras clases las podemos definir como:

$$ clase_{1} = 1 \space si \space \hat{y}(x)\ge 0.5 $$
$$ clase_{0} = 0 \space si \space \hat{y}(x)\lt 0.5 $$

Las clases da igual como las definamos, pueden ser "Sí" y "No" o "No" y "Sí", es decir, simplemente tiene que ser coherente con la pregunta.

¿Es azul? Sí(+1)/No(-1).

¿Es azul? Sí(-1)/No(+1).

En este caso, ¿cuál sería una buena función de pérdida?

¿Usamos el error cuadrático medio como en el caso de la regresión lineal?

![linear to logistic](./imgs/13_linear_vs_logistic_regression.webp)

Lo que queremos son probabilidades que nos digan si algo es más probable que esté en una clase o en otra. Para ello se usa la función logística o sigmoide (por su forma en ese):

$$ S(y_i) = \frac{1}{1+e^{-y_i}} $$

Pero, ¿cómo llegamos a eso?

La probabilidad de que sea 1 debido a sus entradas se puede representar de la siguiente forma:

$$ P(y=1|x) $$

Si pensamos que eso se puede representar de forma lineal y tenemos n dimensiones tendríamos algo como:

$$ P(y=1|x) = w_0 · x_0 + w_1 · x_1 + ... + w_n · x_n $$

($ w_0 = b $ y $ x_0 = 1 $)

El algoritmo de regresión podría ajustar esos pesos pero, como hemos visto en la gráfica de más arriba eso podría llevar a valores que van de $-\infty$ a $\infty$.

El ratio de probabilidades (*odds ratio*) es un término que nos puede ayudar. Se define como la probabilidad de que suceda entre la probabilidad de que no suceda. Como nos encontramos en un caso binario, la probabilidad de que suceda es $p$ y la probabilidad de que no suceda es el resto hasta 1, $1-p$. Su ratio se representa así:

$$ odds(p) = \frac{p}{1-p}$$

Si representamos lo anterior para diferentes valores de $p$ tenemos la siguiente gráfica:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer, make_blobs
from sklearn.metrics import (confusion_matrix, accuracy_score,
                             precision_score, recall_score,
                             roc_curve, roc_auc_score)

from lib.datasets import make_forge
from lib.plots import plot_2d_separator, plot_2d_classification
from lib import discrete_scatter

In [ ]:
x = np.arange(0, 1, 0.01)
odds = x / (1 - x)

fig, ax = plt.subplots()

ax.plot(x, odds)
ax.set_xlabel("$x$")
ax.set_ylabel(r"$\frac{p}{1-p}$");

Si $p=1$ nuestro ratio se dispara lo cual no es lo que queremos.

Podemos usar el logaritmo natural del ratio y nos da lo siguiente:

In [ ]:
x = np.arange(0.01, 1, 0.01)
odds = np.log(x / (1 - x))

fig, ax = plt.subplots()

ax.plot(x, odds)
ax.set_xlabel("$x$")
ax.set_ylabel(r"$\log(\frac{p}{1-p})$");

Como podemos mapear una combinación lineal de entradas arbitrarias a la función del logaritmo natural de los ratios podemos aprovechar eso para tener:

$$ log\_odds(P(y=1|x)) = w_0 · x_0 + w_1 · x_1 + ... + w_n · x_n $$

($ w_0 = b $ y $ x_0 = 1 $)

Si queremos la $ P(y=1|x) $ podemos buscar la inversa del logaritmo natural ($log\_odds$) para obtenerlo. Si nos aprovechamos de algunas identidades de logaritmos y exponenciales:

$$ y = log(\frac{x}{1-x}) $$

$$ x = log(\frac{y}{1-y}) $$

$$ e^x = \frac{y}{1-y} $$

$$y = (1-y)*e^x $$

$$ y = e^x - y*e^x $$

$$ y + ye^x = e^x $$

$$ y*(1 + e^x) = e^x $$

$$ y = \frac{e^x}{1+e^x} $$

$$ y = \frac{1}{\frac{1}{e^x} + 1} $$

$$ y = \frac{1}{1 + e^{-x}} $$

La última expresión es la inversa de $lod\_odds$ y se parece mucho a la función logística o sigmoide (en realidad lo es...):

In [ ]:
x = np.arange(-10, 10, 0.01)
inv_odds = 1 / (1 + np.exp(-x))

fig, ax = plt.subplots()

ax.plot(x, inv_odds)
ax.set_xlabel("$x$")
ax.set_ylabel(r"$\frac{1}{1+e^{-x}}$");

Vaya, ahora todo se restringe a 0 y 1 en el eje *y*.

Por tanto, volviendo al principio teníamos que:

$$ log\_odds(P(y=1|x)) = w_0 · x_0 + w_1 · x_1 + ... + w_n · x_n $$

($ w_0 = b $ y $ x_0 = 1 $)

Si simplificamos la notación tenemos:

$$ log\_odds(P(y=1|x)) = w^T·x $$

Si ahora, finalmente, sacamos la inversa de lo anterior nos quedará:

$$ P(y=1|x) = \frac{1}{1+e^{-w^T·x}} $$

## Estimadores de máxima verosimilitud

En inglés se conocen como *maximum likelihood estimators* (MLE). Es un método para obtener los parámetros de un modelo estadístico. El método obtiene los parámetros buscando los valores de los parámetros que maximizan la función de verosimilitud. Las estimaciones se conocen como estimadores de máxima verosimilitud.

Veamos como funciona esto en la práctica. Para ello vamos a usar la distribución normal:

$$ f(x|\mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} exp(\frac{-(x-\mu)^2}{2\sigma^2})  $$

La función de verosimilitud se obtiene de la siguiente forma:

$$ L(\mu, \sigma^2) = f(x_1, x:2,...,x_N|\mu, \sigma^2) = \prod_{i=1}^N f(x_i|\mu, \sigma^2) $$
$$ L(\mu, \sigma^2) = (\frac{1}{2\pi\sigma^2})^{(n/2)} exp(\frac{-\sum_{i=1}^N(x_i-\mu)^2}{2\sigma^2})$$

Lo que queremos hacer es maximizar lo anterior. En la expresión anterior tenemos multiplicaciones pero lo que se suele hacer normalmente es usar el logaritmo de la función de verosimilitud. Y en lugar de maximizar se minimiza usando el negativo del logaritmo de la función de verosimilitud:

$$ ll(\mu, \sigma^2) = \log(L(\mu, \sigma^2)) $$

$$ ll(\mu, \sigma^2) = \log((\frac{1}{2\pi\sigma^2})^{(n/2)} exp(\frac{-\sum_{i=1}^N(x_i-\mu)^2}{2\sigma^2})) $$

$$ nll(\mu, \sigma^2) = -\log(L(\mu, \sigma^2)) $$

$$ nll(\mu, \sigma^2) = -\log((\frac{1}{2\pi\sigma^2})^{(n/2)} exp(\frac{-\sum_{i=1}^N(x_i-\mu)^2}{2\sigma^2})) $$

**[INCISO]** Un breve paréntesis para anotar un par de propiedades de exponenciales y de logaritmos naturales:

$$ e^x · e ^y = e^{(x + y)} $$

$$ ln(x·y) = ln(x) + ln(y) $$

$$ ln(x/y) = ln(x) - ln(y) $$

$$ ln(x^y) = y·ln(x) $$

Seguimos con MLE...

Si desarrollamos la expresión anterior de $nll$ usando las propiedades de exponenciales y logaritmos nos queda:

$$ nll(\mu, \sigma^2) = -\log((\frac{1}{2\pi\sigma^2})^{(n/2)}) - (\frac{-\sum_{i=1}^N(x_i-\mu)^2}{2\sigma^2}) $$

$$ nll(\mu, \sigma^2) = -\frac{n}{2}\log(\frac{1}{2\pi\sigma^2}) - (\frac{-\sum_{i=1}^N(x_i-\mu)^2}{2\sigma^2}) $$

Para minimizar hemos de usar el gradiente e igualarlo a 0. Si derivamos con respecto a $\mu$:

$$ \frac{\partial{nll(\mu, \sigma^2)}}{\partial{\mu}} = 0 $$

$$ \frac{\partial{nll(\mu, \sigma^2)}}{\partial{\mu}} = \frac{\sum_{i=0}^{N}\mu -\sum_{i=0}^{N}x_i}{2\sigma^2}  = 0 $$

$$ \sum_{i=0}^{N}\mu -\sum_{i=0}^{N}x_i = 0 $$

$$ n·\mu -\sum_{i=0}^{N}x_i = 0 $$

$$ \mu = \frac{\sum_{i=0}^{N}x_i}{n} $$

De la misma forma, derivamos $nll$ con respecto a $\sigma$ e igualamos a 0:

$$ \frac{\partial{nll(\mu, \sigma^2)}}{\partial{\sigma}} = 0 $$

$$ \frac{\partial{nll(\mu, \sigma^2)}}{\partial{\sigma}} = \frac{n}{\sigma} - \frac{\sum_{i=1}^{N}(x_i-\mu)^2}{\sigma^3}= 0 $$

$$ \sigma^2 = \sum_{i=1}^{N}\frac{(x_i-\mu)^2}{n} $$

De esta forma obtendríamos los parámetros de la distribución normal.

Vamos a aplicar lo mismo ahora para el caso de la regresión logística:



La función de verosimilitud será (para una clasificación binaria) para un caso (un ejemplo $(x_i, y_i)$) será:

$$ L(\textbf{w})_i = p(x_i|\textbf{w})^{y_i} · (1-p(x_i|\textbf{w})^{1-y_i}) $$

Si lo generalizamos para los n casos (datos) tenemos la función de verosimilitud:

$$ L(\textbf{w}) = \prod_{i=1}^{N}p(x_i|\textbf{w})^{y_i} · (1-p(x_i|\textbf{w})^{1-y_i}) $$

El *log-likelihood* será:

$$ ll(\textbf{w}) = \sum_{i=1}^{N}({y_i}·\log(p(x_i|\textbf{w})) + {1-y_i}·\log(1-p(x_i|\textbf{w}))) $$

Si toqueteamos lo anterior llegaremos a lo siguiente (saltando pasos):

$$ ll = \sum_{i=1}^{N}y_{i}\textbf{w}^{T}x_{i} - log(1+e^{\textbf{w}^{T}x_{i}}) $$

Si de lo anterior sacamos el gradiente (saltamos pasos):

$$ \bigtriangledown ll = X^{T}(Y - \hat{Y}) $$

El gradiente del *log-likelihood* no es más que la multiplicación de la traspuesta de las entradas por el error de la predicción.

## Implementación de la regresión logística

Con toda esta chicha vamos a construir nuestro algoritmo de regresión logística:

In [ ]:
class RegLog:
    
    def __init__(self, learning_rate=0.001, num_iters=50_000):
        self.X = None
        self.y = None
        self.lr = learning_rate
        self.ni = num_iters
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def _loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, x, y):
        # weights initialization
        intercept = np.ones((x.shape[0], 1))
        self.X = np.concatenate((intercept, x), axis=1)
        self.y = y
        self.w = np.zeros(self.X.shape[1])
        
        for i in range(self.ni):
            z = np.dot(self.X, self.w)
            h = self._sigmoid(z)
            gradient = np.dot(self.X.T, (h - y)) / y.size
            self.w -= self.lr * gradient
            
            z = np.dot(self.X, self.w)
            h = self._sigmoid(z)
            loss = self._loss(h, self.y)
                
    def predict_prob(self, x):
        intercept = np.ones((x.shape[0], 1))
        x = np.concatenate((intercept, x), axis=1)
        return self._sigmoid(np.dot(x, self.w))
    
    def predict(self, x):
        return self.predict_prob(x).round()

Vamos a ver qué tal funciona. Obtenemos unos datos:

In [ ]:
X, y = make_forge()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

Los pintamos para ver cómo son:

In [ ]:
fig, ax = plt.subplots()

idx = y_train == 0
ax.scatter(X_train[idx, 0], X_train[idx, 1], color='b', label="clase 0")
ax.scatter(X_train[~idx, 0], X_train[~idx, 1], color='r', label="clase 1")
ax.legend()
idx = y_test == 0
ax.scatter(X_test[idx, 0], X_test[idx, 1], color='b', s=200)
ax.scatter(X_test[~idx, 0], X_test[~idx, 1], color='r', s=200)

Instanciamos nuestra clase recien creada.

In [ ]:
reglog = RegLog()

Ajustamos los datos de entrenamiento:

In [ ]:
reglog.fit(X_train, y_train)

Predecimos con nuestro modelo ajustado los valores de prueba para ver si se parecen a las etiquetas:

In [ ]:
y_pred = reglog.predict(X_test)

In [ ]:
for yi, yyi, x0, x1 in zip(y_pred, y_test, X_test[:,0], X_test[:,1]):
    print(int(yi), yyi, x0, x1)

Vamos a comparar con lo que trae `scikit-learn`. Instanciamos:

In [ ]:
logreg = LogisticRegression(C=1e20, fit_intercept=False)

Ajustamos:

In [ ]:
logreg.fit(X_train, y_train)

Predecimos:

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
for yi, yyi in zip(y_pred, y_test):
    print(int(yi), yyi)

Parece que lo que obtenemos es similar a nuestro algoritmo de más arriba. Vamos a ver los pesos que obtenemos:

In [ ]:
print(reglog.w)

In [ ]:
print(logreg.intercept_, logreg.coef_)

Se parecen bastante. Podéis modificar la tasa de aprendizaje o el número de iteraciones para ver si mejora o empeora.

## Más detalles y cosas interesantes

En la instancia de más arriba de `LogisticRegression` estoy usando un parámetro que le llama `C`. Este hiperparámetro es el que regula la regularización. Por defecto, la regresión logística en `scikit-learn` usa regularización L2 y valores más altos de `C` llevan a quitarle importancia a la regularización (al revés que con el $\alpha$ de la regressión *Ridge* o la regresión Lasso). Por eso, en el ejemplo de más arriba he usado `C=1e20` para que la regularización fuese poco importante ya que en nuestro algoritmo no hemos incluido regularización para simplificar las cosas...

Vamos a usar el modelo que implementa `scikit-learn` con los valores por defecto:

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train, y_train)

Varias cosas de las que se ven ahí arriba. 

* El *solver* es lo que realiza la optimización. Podéis leer más sobre ello aquí: https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html. En el caso de scikit-learn usa *solvers* especializados. Más info aquí: https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

* `C` es el parámetro que regula la regularización. Un valor bajo regulariza mucho mientras que un valor muy alto le quita importancia a la regularización.

* `class_weight` nos permite dar más peso a alguna clase (acordáos de los datos desnivelados o descompensados (*imbalanced datasets*).

* `penalty` define el tipo de regularización. Dependiendo del *solver* se podrán usar unos tipos de regularizaciones u otros.

* ...

Como veis, hay muchas cosas que se pueden toquetear y que llevarán a resultados diferentes. Dependiendo de los datos que tengamos tendrá sentido usar unas cosas u otras.

Veamos como afecta el parámetro `C`:

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15, 5))
for i, C in enumerate([0.001, 0.01, 1, 100, 10000]):
    logreg = LogisticRegression(C=C).fit(X, y)
    plot_2d_separator(logreg, X, fill=False, eps=0.5, ax=axs[i], alpha=.7)
    discrete_scatter(X[:, 0], X[:, 1], y, ax=axs[i])
    axs[i].set_title(f"C={C}")
    axs[i].set_xlabel("Clase 0")
    axs[i].set_ylabel("Clase 1")
    axs[i].legend()

Por mucho que nos empeñemos, en el conjunto de datos anterior con una separación lineal es imposible tenerlo todo completamente separado y puede parecer muy limitado. Cuando aumenta el número de dimensiones es cuando estos modelos pueden adquirir más importancia. Vamos a usar los datos *Cancer*:

In [ ]:
cancer = load_breast_cancer()
print(cancer.DESCR)

In [ ]:
print(cancer.data.shape)

In [ ]:
print(cancer.target.shape)

In [ ]:
print(cancer.feature_names)

In [ ]:
print(cancer.target_names)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, 
    cancer.target, 
    stratify=cancer.target, 
    random_state=42
)

En lo anterior usamos la *keyword* `stratify` para balancear la misma proporción de etiquetas en los datos de entrenamiento y de prueba, es decir que tengan la misma proporción de etiquetas en ambos casos.

In [ ]:
print(y_train.sum() / len(y_train), y_test.sum() / len(y_test))

Instanciamos el modelo y mostramos qué tal se comporta:

In [ ]:
logreg = LogisticRegression().fit(X_train, y_train)
print(logreg.score(X_train, y_train))
print(logreg.score(X_test, y_test))

Parece que el algoritmo va muy bien. El hecho de que el *score* en ambos casos esté muy cerca y en la prueba esté dando levemente mejor que en el entrenamiento quizá sea indicativo de que estamos infraestimando. Vamos a tocar el valor de `C` para que regularice menos:

In [ ]:
logreg100 = LogisticRegression(C=100).fit(X_train, y_train)
print(logreg100.score(X_train, y_train))
print(logreg100.score(X_test, y_test))

En este caso conseguimos subir el valor de *score* y en el entrenamiento es levemente superior.

Si hacemos lo mismo pero ahora regularizando más que en el valor por defecto veamos a ver qué sale:

In [ ]:
logreg001 = LogisticRegression(C=0.001).fit(X_train, y_train)
print(logreg001.score(X_train, y_train))
print(logreg001.score(X_test, y_test))

En este caso los valores son más bajos. Tenemos un modelo demasiado simple.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

ax.plot(logreg.coef_.T, 'ko', label="C=1")
ax.plot(logreg100.coef_.T, 'b^', label="C=100")
ax.plot(logreg001.coef_.T, 'yv', label="C=0.001")
ax.set_xticks(range(cancer.data.shape[1]))
ax.set_xticklabels(cancer.feature_names, rotation=90)
ax.hlines(0, 0, cancer.data.shape[1])
ax.set_ylim(-5, 5)
ax.set_xlabel("Coeficiente")
ax.set_ylabel("Magnitud del coeficiente")
ax.legend()

Si queremos un modelo más interpretable, como vimos anteriormente, quizá nos interese usar una regularización L1 ya que vemos que muchas dimensiones están cercanas a 0 independientemente de la magnitud de la regularización L2 que usemos. Esto nos puede estar indicando que quizá sea conveniente deshacerse de algo de "ruido".

Vamos a hacer lo mismo que antes pero con regularización L1:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

for C, marker_set in zip([0.001, 1, 100], ['ko', 'b^', 'yv']):
    lr_l1 = LogisticRegression(C=C, penalty="l1").fit(X_train, y_train)
    print("Train", C, lr_l1.score(X_train, y_train))
    print("Test ", C, lr_l1.score(X_test, y_test))
    ax.plot(lr_l1.coef_.T, marker_set, label=f"C={C:.3f}")
    ax.set_xticks(range(cancer.data.shape[1]))
    ax.set_xticklabels(cancer.feature_names, rotation=90)
    ax.hlines(0, 0, cancer.data.shape[1])
    ax.set_xlabel("Coeficiente")
    ax.set_ylabel("Magnitud del coeficiente")
    ax.set_ylim(-5, 5)
ax.legend(loc=3)

Como se puede ver, los algoritmos de clasificación lineales guardan muchas similitudes con los algoritmos de regresión lineales.

## Evaluación del modelo

Ya hemos hablado de la matriz de confusión. Vamos a ver como se ven los modelos usando la matriz de confusión:

In [ ]:
logreg01 = LogisticRegression(C=0.1).fit(X_train, y_train)
logreg10 = LogisticRegression(C=10).fit(X_train, y_train)

y_pred01 = logreg01.predict(X_test)
y_pred10 = logreg10.predict(X_test)

mc01 = confusion_matrix(y_test, y_pred01)
mc10 = confusion_matrix(y_test, y_pred10)

In [ ]:
print(mc01)
print(mc10)

In [ ]:
print(cancer.target[0], cancer.target_names[0])

Como vemos, la clase 0 se asigna a los cánceres malignos. Sabiendo esto vamos a dibujar la matriz de confusión:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12,5))

axs[0].set_xticks([0, 1])
axs[0].set_yticks([0, 1])
sns.heatmap(pd.DataFrame(mc01), annot=True, cmap="YlGnBu" ,fmt='g', ax=axs[0])
axs[0].xaxis.set_label_position("top")
axs[0].set_title('Matriz de Confusión (C=0.01)', y=1.1)
axs[0].set_ylabel('Actual label')
axs[0].set_xlabel('Predicted label')

axs[1].set_xticks([0, 1])
axs[1].set_yticks([0, 1])
sns.heatmap(pd.DataFrame(mc10), annot=True, cmap="YlGnBu" ,fmt='g', ax=axs[1])
axs[1].xaxis.set_label_position("top")
axs[1].set_title('Matriz de Confusión (C=10)', y=1.1)
axs[1].set_ylabel('Actual label')
axs[1].set_xlabel('Predicted label')

fig.tight_layout()

En el caso anterior hemos conseguido, regularizando menos, reducir el número de falsos negativos. En casos como estos, donde un falso positivo requeriría hacer más pruebas y luego descartar el problema, un falso negativo es un problema mucho más grave ya que se puede mandar a una persona a casa con un cáncer maligno habiéndole dicho que está todo correcto. En estos casos hemos de ver qué es lo que nos interesa optimizar mejor. Igual nos da un poco más igual tener falsos positivos pero es inaceptable tener falsos negativos.

Vamos a analizar más métricas:

In [ ]:
print("Caso C=0.01")
print("Accuracy:",accuracy_score(y_test, y_pred01))
print("Precision:",precision_score(y_test, y_pred01))
print("Recall:",recall_score(y_test, y_pred01))

print("Caso C=10")
print("Accuracy:",accuracy_score(y_test, y_pred10))
print("Precision:",precision_score(y_test, y_pred10))
print("Recall:",recall_score(y_test, y_pred10))

En el caso anterior nos interesaría que nuestras métricas optimizasen el caso de evitar mandar a gente enferma a casa ya que le dejar a sanos en el hospital puede ser un incordio más aceptable.

Existen más métricas interesantes. Podéis mirar en:

* https://scikit-learn.org/stable/modules/model_evaluation.html#model-evaluation

* https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

## Clasificación multiclase

Hay algoritmos de clasificación que no permiten la clasificación multiclase. La regresión logística es una excepción. Una técnica común para poder usar un algoritmo que solo permite clasificación binaria a un algoritmo con capacidad de clasificación multiclase es usar la estrategia "Uno contra el Resto" (*One-Vs-Rest*). La estrategia "Uno contra el Resto" no es más que etiquetar la clase que queremos clasificar (por ejemplo con un 1) contra el resto de clases (todas tendrán clase, por ejemplo, 0). De tal forma que tendremos tantos modelos de clasificación binaria como clases queramos clasificar. Para hacer una predicción todos los clasificadores binarios se ejecutan usando un punto de prueba. El clasificador que tiene el *score* más alto en su clase será el "ganador" y esa clase se devuelve como el resultado de la predicción.

Vamos a ver esto con un ejemplo para ver si se entiende mejor:

In [ ]:
X, y = make_blobs(random_state=42)

Vamos a dibujar estos datos a ver cómo se ven:

In [ ]:
fig, ax = plt.subplots()
discrete_scatter(X[:, 0], X[:, 1], y, ax=ax)
ax.set_xlabel("Feature 0")
ax.set_ylabel("Feature 1")
ax.legend(["Class 0", "Class 1", "Class 2"])

Entrenamos una regresión logística:

In [ ]:
logreg = LogisticRegression(multi_class="ovr").fit(X, y)

In [ ]:
print("Coef dims: ", logreg.coef_.shape)
print("Intercept dims: ", logreg.intercept_.shape)

Vemos que está usando 3 clases y dos *features* (lo que tenemos en el *dataset*). Vamos a ver las líneas que define cada uno de los tres conjuntos:

In [ ]:
fig, ax = plt.subplots()

discrete_scatter(X[:, 0], X[:, 1], y, ax=ax)
line = np.linspace(-15, 15)
for coef, intercept, color in zip(logreg.coef_, logreg.intercept_, ['b', 'r', 'g']):
    ax.plot(line, -(line * coef[0] + intercept) / coef[1], c=color)
    ax.set_ylim(-10, 15)
    ax.set_xlim(-10, 8)
    ax.set_xlabel("Feature 0")
    ax.set_ylabel("Feature 1")
ax.legend(['Class 0', 'Class 1', 'Class 2', 
           'Line class 0', 'Line class 1','Line class 2'], 
           loc=(1.01, 0.3))

Cada línea muestra la región donde la clase se define como propia o "resto".

Pero ¿qué pasa con la región del medio que ninguna clase define como propia?

Si un punto cae en ese triángulo, ¿a qué clase pertenecerá?

Pues será la clase que tenga el valor más alto, es decir, la clase con la línea más próxima.

In [ ]:
fig, ax = plt.subplots()

plot_2d_classification(logreg, X, fill=True, alpha=.7, ax=ax)
discrete_scatter(X[:, 0], X[:, 1], y, ax=ax)
line = np.linspace(-15, 15)
for coef, intercept, color in zip(logreg.coef_, logreg.intercept_, ['b', 'r', 'g']):
    ax.plot(line, -(line * coef[0] + intercept) / coef[1], c=color)
    ax.legend(['Class 0', 'Class 1', 'Class 2', 
               'Line class 0', 'Line class 1', 'Line class 2'], 
               loc=(1.01, 0.3))
    ax.set_xlabel("Feature 0")
    ax.set_ylabel("Feature 1")

## Otros modelos lineales de clasificación

Le podéis echar un ojo a `LinearSVC` o a `SGDClassifier` u otros en `sklearn.linear_models`. Del primero comentaremos cosas más adelante. Del segundo comentamos cosas usándolo en regresión.

## Otros apuntes

El principal parámetro de los modelos lineales es el parámetro de regularización ($\alpha$ en los modelos de regresión lineales, como la regresión lineal, *Ridge* o Lasso, y `C` en los modelos de clasificación lineales, como la regresión logística, las máquinas de vectores soporte lineales o el clasificador de Gradiente descendiente estocástico. Otra decisión importante en los modelos lineales es el tipo de regularización que queremos usar, principalmente L1 o L2. Dependiendo del problema nos interesará más una estrategia u otra a la hora de seleccionar estos parámetros.

Los modelos lineales son muy rápidos de entrenar y en la predicción. Escalan muy bien a conjuntos de datos muy grandes. Otra fortaleza de los modelos lineales es que suele ser más fácil de ver cómo se obtienen las predicciones. Desafortunadamente no siempre es sencillo saber porqué los coeficientes son como son. Esto es especialmente importante si tus datos tienen dimensiones altamente correlacionadas (multicolinealidad) y será complicado interpretar los coeficientes.

Los modelos lineales, a menudo, se comportan bien cuando el número de dimensiones es largo comparado con el número de datos. También se usan en casos con gran cantidad de datos, principalmente, porque otros modelos no escalan igual de bien y en muchos casos no es posible usarlos. En casos de baja dimensionalidad otros modelos pueden resultar más interesantes puesto que pueden generalizar mejor.

## Referencias

* http://karlrosaen.com/ml/notebooks/logistic-regression-why-sigmoid/

* Sección 4.4.1 de https://web.stanford.edu/~hastie/ElemStatLearn//printings/ESLII_print12.pdf

* https://github.com/martinpella/logistic-reg/blob/master/logistic_reg.ipynb

* https://beckernick.github.io/logistic-regression-from-scratch/

* https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python